## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, Compute the camera calibration using chessboard images

In [104]:
import numpy as np
import os
import cv2
import glob
import matplotlib.pyplot as plt
import collections
# %matplotlib qt
%matplotlib inline

In [105]:
def calibrate_camera():
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('./camera_cal/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
    
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, (1280,720), None, None)
    return mtx,dist
        

# Apply the correction to a test image


In [106]:
def apply_correction(img, mtx, dist):
    
    undistorted = cv2.undistort(img, mtx, dist, None, mtx)
#     plt.imshow(cv2.cvtColor(undistorted, cv2.COLOR_BGR2RGB))
    return undistorted


# Apply the color tranform, gradient to get the binary image

In [107]:
# def threshold_image(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
#     img = np.copy(img)
#     # Convert to HLS color space and separate the V channel
#     hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
#     l_channel = hls[:,:,1]
#     s_channel = hls[:,:,2]
#     # Sobel x
#     sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0, ksize=9) # Take the derivative in x
#     abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
#     scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
#     # Threshold x gradient
#     sxbinary = np.zeros_like(scaled_sobel)
#     sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
#     # Threshold color channel
#     s_binary = np.zeros_like(s_channel)
#     s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
#     # Stack each channel
# #     color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
#     return s_binary

yellow_HSV_th_min = np.array([0, 70, 70])
yellow_HSV_th_max = np.array([50, 255, 255])


def thresh_frame_in_HSV(frame, min_values, max_values, verbose=False):
    """
    Threshold a color frame in HSV space
    """
    HSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    min_th_ok = np.all(HSV > min_values, axis=2)
    max_th_ok = np.all(HSV < max_values, axis=2)

    out = np.logical_and(min_th_ok, max_th_ok)

    if verbose:
        plt.imshow(out, cmap='gray')
        plt.show()

    return out


def thresh_frame_sobel(frame, kernel_size):
    """
    Apply Sobel edge detection to an input frame, then threshold the result
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=kernel_size)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=kernel_size)

    sobel_mag = np.sqrt(sobel_x ** 2 + sobel_y ** 2)
    sobel_mag = np.uint8(sobel_mag / np.max(sobel_mag) * 255)

    _, sobel_mag = cv2.threshold(sobel_mag, 50, 1, cv2.THRESH_BINARY)

    return sobel_mag.astype(bool)


def get_binary_from_equalized_grayscale(frame):
    """
    Apply histogram equalization to an input frame, threshold it and return the (binary) result.
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    eq_global = cv2.equalizeHist(gray)

    _, th = cv2.threshold(eq_global, thresh=250, maxval=255, type=cv2.THRESH_BINARY)

    return th


def binarize(img, verbose=False):
    """
    Convert an input frame to a binary image which highlight as most as possible the lane-lines.
    :param img: input color frame
    :param verbose: if True, show intermediate results
    :return: binarized frame
    """
    h, w = img.shape[:2]
    binary = np.zeros(shape=(h, w), dtype=np.uint8)

    # highlight yellow lines by threshold in HSV color space
    HSV_yellow_mask = thresh_frame_in_HSV(img, yellow_HSV_th_min, yellow_HSV_th_max, verbose=False)
    binary = np.logical_or(binary, HSV_yellow_mask)

    # highlight white lines by thresholding the equalized frame
    eq_white_mask = get_binary_from_equalized_grayscale(img)
    binary = np.logical_or(binary, eq_white_mask)

    # get Sobel binary mask (thresholded gradients)
    sobel_mask = thresh_frame_sobel(img, kernel_size=9)
    binary = np.logical_or(binary, sobel_mask)

    # apply a light morphology to "fill the gaps" in the binary image
    kernel = np.ones((5, 5), np.uint8)
    closing = cv2.morphologyEx(binary.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
    return closing


# Apply the perspective transform

In [108]:
def warp(img, thresholded_image):
    
    imshape = img.shape
    color=[0, 0, 255]
    thickness=10
    src = np.array([[(200,imshape[0]),
                     ((imshape[1] /2) - 50, (imshape[0] /2) + 100),
                     ((imshape[1] /2) + 50, (imshape[0] /2) + 100),
                     (1100,imshape[0])]], dtype='float32')
    dst = np.array([[(300, imshape[0]),
                     (300,0),
                     ((imshape[1] - 300), 0),
                     ((imshape[1] - 300), imshape[0])]], dtype='float32')



    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    # Warp the image using OpenCV warpPerspective()
    img_size = (imshape[1], imshape[0])
    warped = cv2.warpPerspective(thresholded_image, M, img_size, flags=cv2.INTER_NEAREST)
        # draw the lines
#     cv2.line(img, (200,imshape[0]), (1100,imshape[0]),
#                  color,
#                  thickness)
#     cv2.line(img, (int(imshape[1] /2) - 50, int(imshape[0] /2) + 100),
#                 (int(imshape[1] /2) +50, int(imshape[0] /2) + 100),
#                  color,
#                  thickness)
#     cv2.line(img, (200,imshape[0]), (int(imshape[1] /2) - 50, int(imshape[0] /2) + 100),
#                  color,
#                  thickness)
#     cv2.line(img, (int(imshape[1] /2) +50, int(imshape[0] /2) + 100), (1100,imshape[0]),
#                  color,
#                  thickness)
    return warped, Minv


In [109]:
def find_lane_pixels_by_sliding_window(binary_warped):
    
    global detected
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))

    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    ym_per_pix = 30 / 720   # meters per pixel in y dimension
    xm_per_pix = 3.7 / 700  # meters per pixel in x dimension
    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        #boundaries of the window ###
        win_xleft_low = leftx_current - margin  # Update this
        win_xleft_high = leftx_current + margin  # Update this
        win_xright_low = rightx_current - margin # Update this
        win_xright_high = rightx_current + margin  # Update this
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
            
    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    detected = True
    if not list(leftx) or not list(lefty):
        left_fit_pixel = last_left_line_pixel
        left_fit_meter = last_left_line_meter
        detected = False
    else:
        left_fit_pixel = np.polyfit(lefty, leftx, 2)
        left_fit_meter = np.polyfit(lefty * ym_per_pix, leftx * xm_per_pix, 2)

    if not list(rightx) or not list(righty):
        right_fit_pixel = last_right_line_pixel
        right_fit_meter = last_right_line_meter
        detected = False
    else:
        right_fit_pixel = np.polyfit(righty, rightx, 2)
        right_fit_meter = np.polyfit(righty * ym_per_pix, rightx * xm_per_pix, 2)
    
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    return left_fit_pixel,right_fit_pixel,left_fit_meter,right_fit_meter, out_img

def find_lane_pixels_by_previous_fits(binary_warped,
                                      last_left_line_pixel,
                                      last_right_line_pixel,
                                      last_left_line_meter,
                                      last_right_line_meter):
    """
    Get polynomial coefficients for lane-lines detected in an binary image.
    This function starts from previously detected lane-lines to speed-up the search of lane-lines in the current frame.
    :param birdeye_binary: input bird's eye view binary image
    :param line_lt: left lane-line previously detected
    :param line_rt: left lane-line previously detected
    :param verbose: if True, display intermediate output
    :return: updated lane lines and output image
    """
    global detected
    ym_per_pix = 30 / 720   # meters per pixel in y dimension
    xm_per_pix = 3.7 / 700  # meters per pixel in x dimension

    height, width = binary_warped.shape

    nonzero = binary_warped.nonzero()
    nonzero_y = np.array(nonzero[0])
    nonzero_x = np.array(nonzero[1])
    margin = 100
    left_lane_inds = (
    (nonzero_x > (last_left_line_pixel[0] * (nonzero_y ** 2) + last_left_line_pixel[1] * nonzero_y + last_left_line_pixel[2] - margin)) & (
    nonzero_x < (last_left_line_pixel[0] * (nonzero_y ** 2) + last_left_line_pixel[1] * nonzero_y + last_left_line_pixel[2] + margin)))
    right_lane_inds = (
    (nonzero_x > (last_right_line_pixel[0] * (nonzero_y ** 2) + last_right_line_pixel[1] * nonzero_y + last_right_line_pixel[2] - margin)) & (
    nonzero_x < (last_right_line_pixel[0] * (nonzero_y ** 2) + last_right_line_pixel[1] * nonzero_y + last_right_line_pixel[2] + margin)))

    # Extract left and right line pixel positions
    leftx, lefty = nonzero_x[left_lane_inds], nonzero_y[left_lane_inds]
    rightx, righty = nonzero_x[right_lane_inds], nonzero_y[right_lane_inds]

    detected = True
    if not list(leftx) or not list(lefty):
        left_fit_pixel = last_left_line_pixel
        left_fit_meter = last_left_line_meter
        detected = False
    else:
        left_fit_pixel = np.polyfit(lefty, leftx, 2)
        left_fit_meter = np.polyfit(lefty * ym_per_pix, leftx * xm_per_pix, 2)

    if not list(rightx) or not list(righty):
        right_fit_pixel = last_right_line_pixel
        right_fit_meter = last_right_line_meter
        detected = False
    else:
        right_fit_pixel = np.polyfit(righty, rightx, 2)
        right_fit_meter = np.polyfit(righty * ym_per_pix, rightx * xm_per_pix, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, height - 1, height)
    left_fitx = left_fit_pixel[0] * ploty ** 2 + left_fit_pixel[1] * ploty + left_fit_pixel[2]
    right_fitx = right_fit_pixel[0] * ploty ** 2 + right_fit_pixel[1] * ploty + right_fit_pixel[2]

    # Create an image to draw on and an image to show the selection window
    img_fit = np.dstack((binary_warped, binary_warped, binary_warped)) * 255
    window_img = np.zeros_like(img_fit)

    # Color in left and right line pixels
    img_fit[nonzero_y[left_lane_inds], nonzero_x[left_lane_inds]] = [255, 0, 0]
    img_fit[nonzero_y[right_lane_inds], nonzero_x[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx - margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx + margin, ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx - margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx + margin, ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0, 255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0, 255, 0))
    result = cv2.addWeighted(img_fit, 1, window_img, 0.3, 0)

    return left_fit_pixel,right_fit_pixel,left_fit_meter,right_fit_meter, result

def fit_polynomial(binary_warped, isVideo=True):
    ym_per_pix = 30 / 720   # meters per pixel in y dimension
    xm_per_pix = 3.7 / 700  # meters per pixel in x dimension
    # Find our lane pixels first
    global last_left_line_pixel
    global last_right_line_pixel
    global last_left_line_meter
    global last_right_line_meter
    global recent_left_fits_pixel
    global recent_right_fits_pixel

    if not detected:
        left_fit_pixel, right_fit_pixel,left_fit_meter, right_fit_meter, out_img = find_lane_pixels_by_sliding_window(binary_warped)
    else:
        left_fit_pixel, right_fit_pixel,left_fit_meter, right_fit_meter, out_img = find_lane_pixels_by_previous_fits(binary_warped,last_left_line_pixel, last_right_line_pixel,last_left_line_meter, last_right_line_meter)

    if isVideo:
        last_left_line_pixel=left_fit_pixel
        last_right_line_pixel=right_fit_pixel
        last_left_line_meter=left_fit_meter
        last_right_line_meter=right_fit_meter
        recent_left_fits_pixel.append(left_fit_pixel)
        recent_right_fits_pixel.append(right_fit_pixel)
        recent_left_fits_meter.append(left_fit_meter)
        recent_right_fits_meter.append(right_fit_meter)
    
    return out_img,left_fit_pixel,right_fit_pixel,left_fit_meter,right_fit_meter

In [110]:
def draw(mask, left_fit, right_fit, color=(255, 0, 0), line_width=50, average=False):
    """
    Draw the line on a color mask image.
    """
    h, w, c = mask.shape

    plot_y = np.linspace(0, h - 1, h)
    left_fit = np.mean(recent_left_fits_pixel, axis=0) if average else left_fit
    right_fit = np.mean(recent_right_fits_pixel, axis=0) if average else right_fit

    line_center = left_fit[0] * plot_y ** 2 + left_fit[1] * plot_y + left_fit[2]
    line_left_side = line_center - line_width // 2
    line_right_side = line_center + line_width // 2

    # Some magic here to recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array(list(zip(line_left_side, plot_y)))
    pts_right = np.array(np.flipud(list(zip(line_right_side, plot_y))))
    pts = np.vstack([pts_left, pts_right])
    mask = cv2.fillPoly(mask, [np.int32(pts)], color)
    
    line_center = right_fit[0] * plot_y ** 2 + right_fit[1] * plot_y + right_fit[2]
    line_left_side = line_center - line_width // 2
    line_right_side = line_center + line_width // 2

    # Some magic here to recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array(list(zip(line_left_side, plot_y)))
    pts_right = np.array(np.flipud(list(zip(line_right_side, plot_y))))
    pts = np.vstack([pts_left, pts_right])
    mask = cv2.fillPoly(mask, [np.int32(pts)], color)

    return mask

In [111]:
def draw_back_onto_the_road(img_undistorted, Minv, left_fit, right_fit, average):
    """
    Draw both the drivable lane area and the detected lane-lines onto the original (undistorted) frame.
    :param img_undistorted: original undistorted color frame
    :param Minv: (inverse) perspective transform matrix used to re-project on original frame
    :param line_lt: left lane-line previously detected
    :param line_rt: right lane-line previously detected
    :param keep_state: if True, line state is maintained
    :return: color blend
    """
    height, width, _ = img_undistorted.shape

    left_fit = np.mean(recent_left_fits_pixel, axis=0) if average else left_fit
    right_fit = np.mean(recent_right_fits_pixel, axis=0) if average else right_fit

    # Generate x and y values for plotting
    ploty = np.linspace(0, height - 1, height)
    left_fitx = left_fit[0] * ploty ** 2 + left_fit[1] * ploty + left_fit[2]
    right_fitx = right_fit[0] * ploty ** 2 + right_fit[1] * ploty + right_fit[2]

    # draw road as green polygon on original frame
    road_warp = np.zeros_like(img_undistorted, dtype=np.uint8)
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    cv2.fillPoly(road_warp, np.int_([pts]), (0, 255, 0))
    road_dewarped = cv2.warpPerspective(road_warp, Minv, (width, height))  # Warp back to original image space

    blend_onto_road = cv2.addWeighted(img_undistorted, 1., road_dewarped, 0.3, 0)
#     plt.imshow(cv2.cvtColor(blend_onto_road, cv2.COLOR_BGR2RGB))
    # now separately draw solid lines to highlight them
    line_warp = np.zeros_like(img_undistorted)
    line_warp = draw(line_warp, left_fit, right_fit,color=(255, 0, 0), average=average)
    line_dewarped = cv2.warpPerspective(line_warp, Minv, (width, height))

    lines_mask = blend_onto_road.copy()
    idx = np.any([line_dewarped != 0][0], axis=2)
    lines_mask[idx] = line_dewarped[idx]

    blend_onto_road = cv2.addWeighted(src1=lines_mask, alpha=0.8, src2=blend_onto_road, beta=0.5, gamma=0.)

    return blend_onto_road

In [112]:
def curvature_meter(coeffs):
    y_eval = 0
    return ((1 + (2 * coeffs[0] * y_eval + coeffs[1]) ** 2) ** 1.5) / np.absolute(2 * coeffs[0])

In [113]:
def compute_offset_from_center(left_fit, right_fit, frame_height, frame_width):
    """
    Compute offset from center of the inferred lane.
    The offset from the lane center can be computed under the hypothesis that the camera is fixed
    and mounted in the midpoint of the car roof. In this case, we can approximate the car's deviation
    from the lane center as the distance between the center of the image and the midpoint at the bottom
    of the image of the two lane-lines detected.
    :param left_fit: detected left lane-line
    :param right_fit: detected right lane-line
    :param frame_width: width of the undistorted frame
    :param frame_height: height of the undistorted frame
    :return: inferred offset
    """
    xm_per_pix = 3.7 / 700
    ploty = np.linspace(0, frame_height - 1, frame_height)
    ploty = np.max(ploty)
    line_lt_bottom = left_fit[0] * ploty ** 2 + left_fit[1] * ploty + left_fit[2]
    line_rt_bottom = right_fit[0] * ploty ** 2 + right_fit[1] * ploty + right_fit[2]
    lane_width = line_rt_bottom - line_lt_bottom
    midpoint = frame_width / 2
    offset_pix = abs((line_lt_bottom + lane_width / 2) - midpoint)
    offset_meter = xm_per_pix * offset_pix

    return offset_meter

In [114]:
def calculate_and_write(blend_on_road, left_line_meter, right_line_meter, left_line_pixel, right_line_pixel,average=False):
    """
    Prepare the final output image, given all intermediate pipeline images
    :param blend_on_road: color image of lane blend onto the road
    :param left_line_meter: detected left lane-line in meters
    :param right_line_meter: detected right lane-line in meters
    :param left_line_pixel: detected left lane-line
    :param right_line_pixel: detected right lane-line
    :return: the final output image with all the text written
    """
    h, w = blend_on_road.shape[:2]

    # add text (curvature and offset info) on the upper left of the blend
    left_line_meter = np.mean(recent_left_fits_meter, axis=0) if average else left_line_meter
    right_line_meter = np.mean(recent_right_fits_meter, axis=0) if average else right_line_meter
    left_line_pixel = np.mean(recent_left_fits_pixel, axis=0) if average else left_line_pixel
    right_line_pixel = np.mean(recent_right_fits_pixel, axis=0) if average else right_line_pixel
    curvature_meter_left = curvature_meter(left_line_meter)
    curvature_meter_right = curvature_meter(right_line_meter)
    mean_curvature_meter = np.mean([curvature_meter_left, curvature_meter_right])
    offset_meter = compute_offset_from_center(left_line_pixel, right_line_pixel, h, w)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(blend_on_road, 'Radius of Curvature: {:.02f}m'.format(mean_curvature_meter), (10, 60), font, 0.9, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(blend_on_road, 'Offset from center: {:.02f}m'.format(offset_meter), (10, 100), font, 0.9, (255, 255, 255), 2, cv2.LINE_AA)

    return blend_on_road

In [115]:
def pipeline(test_image, mtx, dist, isVideo=False):
    
    undistorted = apply_correction(test_image, mtx, dist)
    
    thresholded_image = binarize(undistorted)
   
    warped, Minv = warp(undistorted, thresholded_image)

    line_fit_image,left_fit_pixel,right_fit_pixel,left_fit_meter,right_fit_meter = fit_polynomial(warped, isVideo)
    
    lane_image = draw_back_onto_the_road(undistorted, Minv, left_fit_pixel, right_fit_pixel, isVideo)
   
    final_image = calculate_and_write(lane_image, left_fit_meter, right_fit_meter, left_fit_pixel, right_fit_pixel,isVideo)
    return final_image


In [116]:
mtx, dist = calibrate_camera()

In [120]:

test_video = "./project_video.mp4"
output = "./output_video1.mp4"
last_left_line_pixel = np.array([0,0,0])
last_right_line_pixel = np.array([0,0,0])
last_left_line_meter = np.array([0,0,0])
last_right_line_meter = np.array([0,0,0])
recent_left_fits_pixel = collections.deque(maxlen=buffer_len)
recent_right_fits_pixel = collections.deque(maxlen=buffer_len)
recent_left_fits_meter = collections.deque(maxlen=2 * buffer_len)
recent_right_fits_meter = collections.deque(maxlen=2 * buffer_len)
detected=False
cap = cv2.VideoCapture(test_video)
out = cv2.VideoWriter(output,fourcc=cv2.VideoWriter_fourcc(*'DIVX'), fps=25, frameSize=(1280,720))
while cap.isOpened():
    ret, color_frame = cap.read()
    if ret == True:
        out_frame = pipeline(color_frame, mtx, dist,isVideo=True)
        out.write(out_frame)
    else:
        break
        
cap.release()
out.release()

In [118]:

for imageName in os.listdir("./test_images/"):
    buffer_len=10
    last_left_line_pixel = np.array([0,0,0])
    last_right_line_pixel = np.array([0,0,0])
    last_left_line_meter = np.array([0,0,0])
    last_right_line_meter = np.array([0,0,0])
    recent_left_fits_pixel = collections.deque(maxlen=buffer_len)
    recent_right_fits_pixel = collections.deque(maxlen=buffer_len)
    recent_left_fits_meter = collections.deque(maxlen=2 * buffer_len)
    recent_right_fits_meter = collections.deque(maxlen=2 * buffer_len)
    detected=False
    img = cv2.imread("./test_images/" + imageName)
    final_image = pipeline(img, mtx, dist, isVideo=False)
    cv2.imwrite('./output_images/' + imageName, final_image)